<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/KVA_HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 3. ДЗ Embedding word2vec fasttext
**Задача поиск похожих по эмбеддингам**
* на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
* Проверить насколько хорошо работают подходы

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install stop_words

In [ ]:
!pip install annoy

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange, tqdm_notebook

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install corus

In [ ]:
from corus import load_lenta

path = '/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/Lesson03/lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [ ]:
df = pd.DataFrame(columns=['title', 'text'])

In [ ]:
%%time

c = 0

for article in tqdm_notebook(records):
    df = df.append({'title': article.title, 'text': article.text}, ignore_index=True)
    c += 1
    if c > 100000:
        break

<timed exec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The fra

CPU times: user 15min 18s, sys: 13.5 s, total: 15min 32s
Wall time: 16min 29s


<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [ ]:
df.shape

(100001, 2)

In [ ]:
df.to_csv('my_data.csv')

In [ ]:
df = pd.read_csv('my_data.csv')

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
assert True

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
df = df.dropna(subset=['title'])
# df[~df['title'].isnull()]

# обучение на Title

In [ ]:
%%time

c = 0
title_sentences = []

for line in tqdm_notebook(df['title']):
    spls = preprocess_txt(line)
    title_sentences.append(spls)
    c += 1
    if c > 100000:
        break

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 3min 56s, sys: 1.54 s, total: 3min 57s
Wall time: 4min 7s


In [ ]:
%%time

modelW2V_title = Word2Vec(sentences=title_sentences, vector_size=300, window=5, min_count=3)

CPU times: user 18.5 s, sys: 136 ms, total: 18.6 s
Wall time: 12.2 s


In [ ]:
%%time

modelFT_title = FastText(sentences=title_sentences, vector_size=300, min_count=3, window=5, workers=8)

CPU times: user 1min 38s, sys: 1.59 s, total: 1min 39s
Wall time: 1min 4s


In [ ]:
%%time

w2v_index_title = annoy.AnnoyIndex(300 ,'angular')
ft_index_title = annoy.AnnoyIndex(300 ,'angular')

index_map_title = {}
counter = 0

for title in tqdm_notebook(df['title']):
    n_w2v = 0
    n_ft = 0
    index_map_title[counter] = title
    article_morphs = preprocess_txt(title)

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_title.wv:
            vector_w2v += modelW2V_title.wv[word]
            n_w2v += 1
        if word in modelFT_title.wv:
            vector_ft += modelFT_title.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_title.add_item(counter, vector_w2v)
    ft_index_title.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_title.build(10)
ft_index_title.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 4min 50s, sys: 3.68 s, total: 4min 54s
Wall time: 5min


True

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "когда вырастет рубль"

In [ ]:
get_response(TEXT, w2v_index_title, modelW2V_title, index_map_title)

['Официальный евро вырос до\xa061,5\xa0рубля',
 'Курс доллара вырос почти до\xa058\xa0рублей',
 'Рубль рекордно обвалился',
 'Рублю посулили ослабление почти на\xa010\xa0процентов',
 'Евро упал ниже 69\xa0рублей']

In [ ]:
get_response(TEXT, ft_index_title, modelFT_title, index_map_title)

['Доллар поднялся выше 60\xa0рублей',
 'Рубль снова обвалился',
 'Рубль обвалился',
 'Курс доллара вырос почти до\xa058\xa0рублей',
 'Доллар упал ниже 59\xa0рублей']

Обучение на заголовках статей - дает плохой результат.

# обучение на Text

In [ ]:
%%time

c = 0
text_sentences = []

for line in tqdm_notebook(df['text']):
    spls = preprocess_txt(line)
    text_sentences.append(spls)
    c += 1
    if c > 100000:
        break

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 1h 22min 2s, sys: 22.5 s, total: 1h 22min 24s
Wall time: 1h 24min 28s


In [ ]:
%%time

modelW2V_text = Word2Vec(sentences=text_sentences, vector_size=300, window=5, min_count=3)

CPU times: user 6min 40s, sys: 1.62 s, total: 6min 41s
Wall time: 3min 58s


In [ ]:
%%time

modelFT_text = FastText(sentences=text_sentences, vector_size=300, min_count=3, window=5, workers=8)

In [ ]:
%%time

w2v_index_text = annoy.AnnoyIndex(300 ,'angular')
ft_index_text = annoy.AnnoyIndex(300 ,'angular')

index_map_text = {}
counter = 0

for text in tqdm_notebook(df['text']):
    n_w2v = 0
    n_ft = 0
    index_map_text[counter] = text
    article_morphs = preprocess_txt(text)

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_text.wv:
            vector_w2v += modelW2V_text.wv[word]
            n_w2v += 1
        if word in modelFT_text.wv:
            vector_ft += modelFT_text.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_text.add_item(counter, vector_w2v)
    ft_index_text.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_text.build(10)
ft_index_text.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 1h 24min 15s, sys: 24.9 s, total: 1h 24min 40s
Wall time: 1h 26min 40s


True

In [ ]:
TEXT = "когда вырастет рубль"

In [ ]:
index_map_text = index_map

NameError: ignored

In [ ]:
get_response(TEXT, w2v_index_text, modelW2V_text, index_map_text)

In [ ]:
get_response(TEXT, ft_index_text, modelFT_text, index_map_text)

Обучение на тексте статей выдает более-менее похожие статьи - но конечно криво еще работает.